In [13]:
import boto3
from botocore.exceptions import ClientError
from pprint import pprint
import csv
from random import choice

client = boto3.session.Session(profile_name="shared").client('iam')
resource = boto3.session.Session(profile_name="shared").resource('iam')

In [12]:

def list_users():
    """Function to list all iam users
    """
    for each_user in resource.users.all():
        print(each_user.user_name, each_user.tags, each_user.create_date.strftime("%y/%m/%d %H:%M:%S"))
        
        
list_users()        

python None 22/06/22 05:35:16


## Create user with console access.

In [47]:
def get_randome_password():
    password_len = 10
    passwd_li = []
    valid_char = 'b1NDDSJDSd3e30f4dcAHSJDJ!@Be4e2a9d4e8b8e3!@5f5219f%$#)9*hwySGSGWHE'
    for each_char in range(password_len):
        passwd_li.append(choice(valid_char))
    return passwd_li
    
passwd = get_randome_password()
random = "".join(passwd) 
print(random)


D41hyDeSDy


## reduced version of get_random_password

In [72]:
def get_randome_password_v():
    password_len = 10
    passwd_li = []
    valid_char = 'b1NDDSJDSd3e30f4dcAHSJDJ!@Be4e2a9d4e8b8e3!@5f5219f%$#)9*hwySGSGWHE'
    random_passwd =  "".join(choice(valid_char) for each_char in range(password_len))
    return random_passwd

random_password = get_randome_password_v()
print(random_password)

94dy*9%J!f


## Let's  create a user with console access

In [97]:
# this is jupyter notebook
random_password = get_randome_password_v()

def create_user(user_name, passwd=None, policy_arn =None, *args):
    """Creating iam user with console login access
    """
    tag = [{'Key': 'Name','Value': user_name}]
    
    if passwd is None:
        passwd = random_password
    if policy_arn is None:
        policy_arn = 'arn:aws:iam::aws:policy/AdministratorAccess'
    try:
        # Create user
        user_response = client.create_user(UserName= user_name, Tags= tag) 
        
        # create user login with console access
        client.create_login_profile( UserName=user_name,
                                    Password=passwd,
                                    PasswordResetRequired=False)     
        # attach user policy
        client.attach_user_policy(UserName=user_name,
                                  PolicyArn=policy_arn
                                 )
    except client.exceptions.EntityAlreadyExistsException:
        print(f"The user {user_name} already exists\nPlease give another valid name")
    except ClientError as err:
        print(err.response)
    else:
        print(f"user_name: {user_response['User']['UserName']}, User_arn: {user_response['User']['Arn']} User_passwed: {passwd}")
    
    return None    
        
  

        
create_user('python-boto') 

('user_name: python-boto, User_arn: arn:aws:iam::181437319056:user/python-boto '
 'User_passwed: )N9ee99!fh')


## Let's  create a user with console and programatic access and dump info in csv

In [106]:
random_password = get_randome_password_v()

def create_user_access_key(user_name, passwd=None, policy_arn =None, *args):
    """Creating iam user with console login access
    """
    tag = [{'Key': 'Name','Value': user_name}]
    
    if passwd is None:
        passwd = random_password
    if policy_arn is None:
        policy_arn = 'arn:aws:iam::aws:policy/AdministratorAccess'
    try:
        # Create user
        user_response = client.create_user(UserName= user_name, Tags= tag) 
        
        # create user login with console access
        client.create_login_profile( UserName=user_name,
                                    Password=passwd,
                                    PasswordResetRequired=False)     
        # attach user policy
        client.attach_user_policy(UserName=user_name,
                                  PolicyArn=policy_arn
                                 )
        
        # create csv hearder
        csv_header = ["UserName", "AccessKeys", "SecretsKeys", "UserArn", "Password"]
        # Create access key
        accesskey_response = client.create_access_key(UserName= user_name)
        
        with open(f"{user_name}.csv", "w",newline='') as file:
            csv_writer = csv.writer(file)
            # creating my csv hearder
            csv_writer.writerow(csv_header)
            csv_writer.writerow([user_name,accesskey_response['AccessKey'].get('AccessKeyId'),accesskey_response['AccessKey'].get('SecretAccessKey'),user_response['User']['Arn'],passwd])
    
    except client.exceptions.EntityAlreadyExistsException:
        print(f"The user {user_name} already exists\nPlease give another valid name")
    except ClientError as err:
        print(err.response)
   
    return None    
        
username = "kojitech"  

create_user_access_key(username)        


The user kojitech already exists
Please give another valid name
